In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [20]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as T
from torchvision.models import ResNet, resnet18, resnet50
from PIL import Image

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier

D:\app\Miniconda3\envs\adv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
img_preprocess = T.Compose([
            # We use single int value inside a list due to torchscript type restrictions
            T.Resize([256, ]),
            T.CenterCrop(224),
            T.ConvertImageDtype(torch.float),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
class ResnetPredictor(nn.Module):

    def __init__(self, model: ResNet):
        super().__init__()
        self.model = model
        self.transforms = img_preprocess

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            x = self.transforms(x)
            _ = self.model(x)
            y_pred: torch.Tensor = nn.Softmax(dim=1)(_).topk(5)
            clazz = y_pred.values.clone().detach().cpu().tolist()
            confs = y_pred.indices.clone().detach().cpu().tolist()
            # topk = y_pred.topk(5)
            return clazz, confs

D:\app\Miniconda3\envs\adv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
r18 = resnet18(pretrained=True, progress=True).eval().cuda()
r50 = resnet50(pretrained=True, progress=True).eval().cuda()
model_version_mapping = {
    50: r50,
    18: r18
}


D:\app\Miniconda3\envs\adv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
pil_image = Image.open("./imgs/dogs.jpg")
if pil_image.mode == 'RGBA':
    pil_image = pil_image.convert('RGB')
img = img_preprocess(T.ToTensor()(pil_image)).cuda()
img

D:\app\Miniconda3\envs\adv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([[[-1.6311, -1.6510, -1.6329,  ..., -1.3003, -1.2754, -1.2826],
         [-1.5901, -1.6027, -1.5785,  ..., -1.2264, -1.2285, -1.2471],
         [-1.6012, -1.5916, -1.5694,  ..., -1.1251, -1.0994, -1.1400],
         ...,
         [-1.0306, -0.2032,  0.0293,  ..., -1.4527, -1.7057, -1.6515],
         [-0.9232, -0.2295, -0.0811,  ..., -1.4868, -1.5933, -1.6319],
         [-0.9191, -0.2305, -0.1058,  ..., -1.4383, -1.5840, -1.7882]],

        [[-0.9253, -0.8873, -0.8409,  ..., -0.4265, -0.4309, -0.4383],
         [-0.8834, -0.8253, -0.7664,  ..., -0.3498, -0.3593, -0.3814],
         [-0.8514, -0.8117, -0.7747,  ..., -0.2505, -0.2242, -0.2657],
         ...,
         [-0.2841,  0.7239,  1.0147,  ..., -0.6250, -1.1354, -1.3591],
         [-0.1694,  0.7125,  0.9028,  ..., -0.7242, -1.1096, -1.3879],
         [-0.1547,  0.7215,  0.8792,  ..., -0.7285, -1.0549, -1.4414]],

        [[-1.1173, -1.1438, -1.1284,  ..., -0.5601, -0.5473, -0.5546],
         [-1.0407, -1.0771, -1.0637,  ..., -0

In [24]:
predictor = ResnetPredictor(r50).to(0)
res = predictor(img.unsqueeze_(0))

D:\app\Miniconda3\envs\adv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
res

([[0.999515175819397,
   0.00024377662339247763,
   0.0001022979777189903,
   6.603808287763968e-05,
   2.5767878469196148e-05]],
 [[818, 506, 971, 968, 421]])

In [46]:
import torch.optim as optim

def getClassifier(model):
   criterion = nn.CrossEntropyLoss()
   optimizer = optim.Adam(model.parameters(), lr=0.01)
   classifier = PyTorchClassifier(
        model=model,
        clip_values=(0, 1),
        loss=criterion,
        optimizer=optimizer,
        input_shape=(3, 224, 224),
        nb_classes=1000,
        )
   return classifier

In [45]:
classifier = getClassifier(r18)

In [43]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    attack = FastGradientMethod(estimator=classifier, eps=0.6)

In [42]:
import numpy as np
x:np.ndarray = img.clone().detach().cpu().numpy()

In [40]:
adv = attack.generate(x)

In [41]:
import kornia
kornia_img = kornia.enhance.denormalize(torch.Tensor(adv), torch.Tensor([0.485, 0.456, 0.406]).unsqueeze_(0), torch.tensor([0.229, 0.224, 0.225]).unsqueeze_(0))


In [19]:
# adv = adv[0]
a = kornia_img.numpy()[0]
# print(a)
import matplotlib.pyplot as plt
import numpy as np
plt.figure()
plt.axis("off")
plt.imshow(a.transpose(1,2,0))
plt.show()

D:\app\Miniconda3\envs\adv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ImportError: DLL load failed while importing ft2font: The specified module could not be found.

In [56]:
predictor = ResnetPredictor(r50).to(0)
res = predictor(kornia_img.cuda())

D:\app\Miniconda3\envs\adv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [57]:
res

D:\app\Miniconda3\envs\adv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


([[0.8273826837539673,
   0.040224142372608185,
   0.032656654715538025,
   0.02755838818848133,
   0.00962915737181902]],
 [[257, 222, 229, 216, 208]])

In [33]:
np.version.version

D:\app\Miniconda3\envs\adv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'1.20.2'